<a href="https://colab.research.google.com/github/a23956491z/deep-learning-research/blob/main/colab/pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print(
    'hi'
)

hi


In [9]:
import torch

# value
t1 = torch.tensor(4.)
# vector
t2 = torch.tensor([1., 2, 3, 4])
# matrix 
t3 = torch.tensor([[5, 6], 
                   [7, 8], 
                   [9, 10]])

# size of tensor
print(t3.shape)

torch.Size([3, 2])

In [15]:
x = torch.tensor(3.)# doesn't have grad
w = torch.tensor(4., requires_grad=True)
b = torch.tensor(5., requires_grad=True)

y = w * x + b
print(y)

# The derivatie of y
y.backward()

print('dy/dx:', x.grad) # doesn't have grad
print('dy/dw:', w.grad)
print('dy/db:', b.grad)

tensor(17., grad_fn=<AddBackward0>)
dy/dx: None
dy/dw: tensor(3.)
dy/db: tensor(1.)


In [19]:
import numpy as np

x = np.array([[1,2], [3,4]])
y = torch.from_numpy(x)

print(x.dtype, y.dtype)

z = y.numpy()
print(z)

int64 torch.int64
[[1 2]
 [3 4]]


In [ ]:
# input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70]], dtype='float32')

# targets (apples, oranges)
targets = np.array([[56, 70],
                    [81, 101],
                    [119, 133],
                    [22, 37],
                    [103, 119]], dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)

def model(x):
  # .t() return transpose
  return x @ w.t() + b

def mse(t1, t2):
  diff = t1-t2
  return torch.sum(diff*diff) / diff.numel()

preds = model(inputs)
loss = mse(preds, targets)


loss.backward()
print(w.grad)
print(b.grad)

for i in range(100):
  preds = model(inputs)
  loss = mse(preds, targets)
  loss.backward()
  print(loss)
  with torch.no_grad():
    w -= w.grad * 1e-4
    b -= b.grad * 1e-4
    w.grad.zero_()
    b.grad.zero_()



preds = model(inputs)
print(preds)

In [61]:
import torch.nn as nn

from torch.utils.data import TensorDataset

train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

from torch.utils.data import DataLoader
batch_size = 3
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

model = nn.Linear(3,2)


print(list(model.parameters()))

preds = model(inputs)

import torch.nn.functional as F
loss_fn = F.mse_loss
loss = loss_fn(model(inputs), targets)
print(loss)

opt = torch.optim.SGD(model.parameters(), lr=1e-5)
def fit(num_epochs, model, loss_fn, opt):
  for epoch in range(num_epochs):

    for xb, yb in train_dl:
      pred = model(xb)
      loss = loss_fn(pred,yb)
      loss.backward()
      opt.step()
      opt.zero_grad()
    
    if(epoch+1) % 10 == 0:
      print("Epoch [{:>3} / {}], Loss: {:.4f}".format(epoch+1, num_epochs, loss))

fit(100, model, loss_fn, opt)

[Parameter containing:
tensor([[-0.4740,  0.3844,  0.4024],
        [ 0.1066, -0.2387,  0.4897]], requires_grad=True), Parameter containing:
tensor([-0.5043, -0.1468], requires_grad=True)]
tensor(5706.4141, grad_fn=<MseLossBackward>)
Epoch [ 10 / 100], Loss: 533.7190
Epoch [ 20 / 100], Loss: 416.4494
Epoch [ 30 / 100], Loss: 75.0797
Epoch [ 40 / 100], Loss: 76.8041
Epoch [ 50 / 100], Loss: 32.1490
Epoch [ 60 / 100], Loss: 83.6493
Epoch [ 70 / 100], Loss: 116.7641
Epoch [ 80 / 100], Loss: 34.4732
Epoch [ 90 / 100], Loss: 70.6477
Epoch [100 / 100], Loss: 3.5452
